<a href="https://colab.research.google.com/github/beatricebianchi3/FINTECH-ML---Driven-early-warning-system/blob/main/CrashBustersApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

In [33]:
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

def randomForest_run(X_train, Y_train, X_val, Y_val):

  # Training
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, Y_train)

  # Model evaluation on validation
  Y_val_pred = model.predict(X_val)
  val_accuracy = accuracy_score(Y_val, Y_val_pred)

  # Get the predicted probabilities for the positive class (Y=1)
  Y_val_probs = model.predict_proba(X_val)[:, 1]

  return Y_val_pred, model


def kNN_run(X_train, Y_train, X_val, Y_val):

  kNN = KNeighborsClassifier()

  # Define a value set for the hyperparameter k
  param_grid = {'n_neighbors': [3, 4, 5, 6, 7, 8]}

  # GridSearch cross-validation for hyperparameter tuning
  grid_search = GridSearchCV(kNN, param_grid, cv=5)
  grid_search.fit(X_train, Y_train)

  # Training

  model = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'])
  model.fit(X_train, Y_train)

  # Model evaluation

  # Evaluate on validation set
  Y_val_pred = model.predict(X_val)
  val_accuracy = accuracy_score(Y_val, Y_val_pred)

  return Y_val_pred, model


# Sample function to simulate model prediction (replace with your actual model prediction function)
def predict_market_crash(file):
    df= pd.read_excel(file.name, sheet_name=2)
    y = df.iloc[:, 0]

    df.drop(df.columns[0], axis=1, inplace=True)
    df.drop(df.columns[0], axis=1, inplace=True)

    # Standardization
    scaler = StandardScaler()
    df_standardized = scaler.fit_transform(df)
    column_names = df.columns
    column_names = column_names.tolist()

    # Create standardized dataframe
    df_standardized_pd = pd.DataFrame(df_standardized, columns=column_names)

    X_train_temp, X_test, y_train_temp, y_test = train_test_split(df_standardized_pd, y, test_size=0.2, random_state=41)
    X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=0.25, random_state=41)

    bond_train = X_train.iloc[:, 9:35]
    equity_train = X_train.iloc[:, 35:42]
    commodities_train = X_train.iloc[:, [0,3,7]]
    indexes_train = X_train.iloc[:, [1,2,4,5,6,8]]

    bond_val = X_val.iloc[:, 9:35]
    equity_val = X_val.iloc[:, 35:42]
    commodities_val = X_val.iloc[:, [0,3,7]]
    indexes_val = X_val.iloc[:, [1,2,4,5,6,8]]

    bond_test = X_test.iloc[:, 9:35]
    equity_test = X_test.iloc[:, 35:42]
    commodities_test = X_test.iloc[:, [0,3,7]]
    indexes_test = X_test.iloc[:, [1,2,4,5,6,8]]

    val_pred_bonds, rf_bonds = randomForest_run(bond_train, y_train, bond_val, y_val)
    val_pred_equity, kNN_equity = kNN_run(equity_train, y_train, equity_val, y_val)
    val_pred_commodities, kNN_commodities = kNN_run(commodities_train, y_train, commodities_val, y_val)
    val_pred_indexes, rf_indexes = randomForest_run(indexes_train, y_train, indexes_val, y_val)

    meta_learner = LogisticRegression(fit_intercept=False)

    X_val_meta = np.column_stack((val_pred_bonds, val_pred_equity, val_pred_commodities, val_pred_indexes))
    meta_learner.fit(X_val_meta, y_val)

    coefficients = meta_learner.coef_

    coefficients_str = (
        "β1 bond:       {:>15.8f}\n".format(coefficients[0][0]) +
        "β2 equity:     {:>15.8f}\n".format(coefficients[0][1]) +
        "β3 commodities:{:>15.8f}\n".format(coefficients[0][2]) +
        "β4 indexes:    {:>15.8f}\n".format(coefficients[0][3])
    )

    test_pred_bonds = rf_bonds.predict(bond_test)
    test_pred_equity = kNN_equity.predict(equity_test)
    test_pred_commodities = kNN_commodities.predict(commodities_test)
    test_pred_indexes = rf_indexes.predict(indexes_test)

    X_test_meta = np.column_stack((test_pred_bonds, test_pred_equity, test_pred_commodities, test_pred_indexes))
    stacking_predictions = meta_learner.predict(X_test_meta)

    predicted_proba = meta_learner.predict_proba(X_test_meta)
    predicted_proba = (predicted_proba >= 0.6).astype(int)
    crashes_prob = predicted_proba[:, 1]

    test_accuracy = accuracy_score(y_test, stacking_predictions)

    classification_report_str = classification_report(y_test,stacking_predictions)

    stacking_predictions_df = pd.DataFrame(stacking_predictions, columns=["Market Crash Predictions"])

    return coefficients_str, classification_report_str, stacking_predictions_df


custom_css = """
body {
    background-color: #f5f5f5;
    font-family: Arial, sans-serif;
}

.gradio-container {
    background-color: #ffffff;
    border: 1px solid #ddd;
    border-radius: 10px;
    padding: 20px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}

.gradio-title {
    color: #333;
    text-align: center;
    font-size: 24px;
    margin-bottom: 20px;
}

.gradio-description {
    color: #666;
    text-align: center;
    margin-bottom: 20px;
}

.gradio-inputs, .gradio-outputs {
    margin-bottom: 20px;
}

.gradio-output-column {
    width: 33.33%;
    display: inline-block;
    vertical-align: top;
}
"""

iface = gr.Interface(
    fn=predict_market_crash,
    inputs=gr.File(label="Upload Market Data"),
    outputs=[
        gr.Textbox(label="Model Coefficients"),
        gr.Textbox(label="Performance Report"),
        gr.Dataframe(label="Market Crash Predictions"),
    ],
    title="CrashBusters",
    description="""
    ### Early Warning System for Market Anomaly Detection

    This application allows you to upload an Excel file containing financial market data to get market crash predictions, model coefficients,
    and a performance report.

    **Instructions:**
    1. Upload an Excel file with your financial data.
    2. The system will process the file and display the results.
    3. Ensure your file has the correct format (market indices as feature columns on the 3rd page).

    **Example:**
    - Column 1: GTITL30YR data
    - Column 2: MXEU data
    - Column 3: XAU BGNL data
    - Column 4: VIX data

    **Note:** The system uses trained machine learning models to predict market crashes.
    """,
    live=True,
    css=custom_css,
)

# Launch the interface
iface.launch(debug=False, share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6ce2abbcb74276c1fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
